In [2]:
!pip install schedule  

In [24]:
import requests  
import json  
import datetime  
import time  
import schedule  
import pandas as pd

url = 'https://kudago.com/public-api/v1.4/events/'  
params = '?order_by=-publication_date&page_size=100&location=msk&fields=id,title,dates,description,price,site_url'  
response = requests.get(url+params)  
results = json.loads(response.text)['results']  
results[0]

{'id': 189509,
 'dates': [{'start': 1606640400, 'end': 1606640400},
  {'start': 1606654800, 'end': 1606654800}],
 'title': 'спектакль «Сказка о царе Салтане» с Александром Олешко в Театре имени Вахтангова',
 'description': '<p>Вахтанговский театр пополнил свой сказочный репертуар ещё одним замечательным моноспектаклем.</p>',
 'price': 'от 300 до 1300 руб.',
 'site_url': 'https://kudago.com/msk/event/teatr-skazka-o-tsare-saltane-noyabr-2020/'}

 Задание 1
1 point possible (graded)
Создайте функцию clear_description(description), которая принимает на входе строковую величину — описание мероприятия, включающее теги начала и окончания абзаца, а возвращает тот же текст без тегов.


In [12]:
#Вариант1
description=results[0]['description']
print(description[3:-4])

В рамках дебютного гастрольного тура по России сенсация отечественного рэпа Еverthe8 поставит на уши московский клуб.


In [16]:
#Вариант2
def clear_description(description):
    description = description.replace('<p>', '')
    description = description.replace('</p>', '')
    return description
desc=clear_description(description)
print(desc)

В рамках дебютного гастрольного тура по России сенсация отечественного рэпа Еverthe8 поставит на уши московский клуб.


Задание 3
1 point possible (graded)

Создайте функцию fill_df(event), которая принимает словарь, описывающий мероприятие в формате API KudaGo, а возвращает словарь, который можно будет добавить в новую строку создаваемого датафрейма

Важно!

Обратите внимание, что внутри функции fill_df() должны вызываться две уже созданные нами функции: clear_description() и find_dates(). Для корректной проверки кода необходимо добавить код обеих функций в поле для ответа вместе с функцией fill_df(). Таким образом, вам необходимо поместить в поле для ответа код трёх функций: clear_description(), find_dates() и fill_df().


In [ ]:

def find_dates(event):  
    startdate_dt = datetime.datetime.fromtimestamp(event['dates'][0]['start'])  
    enddate_dt = datetime.datetime.fromtimestamp(event['dates'][0]['end'])  
    startdate_txt = startdate_dt.strftime('%d-%m-%Y')  
    enddate_txt = enddate_dt.strftime('%d-%m-%Y')  
    return startdate_dt, enddate_dt, startdate_txt, enddate_txt  


df = pd.DataFrame(columns = 'id title description price start_date end_date url'.split())
for event in results:     
    # С помощью функции find_dates получаем данные о дате начала и окончания мероприятия в формате datetime  
    # т.к. даты в текстовом формате нам пока не нужны, оставляем на месте соответствующих переменных прочерки  
    start_dt, end_dt, _, _ = find_dates(event)  
    now = datetime.datetime.now()  
    # Проверяем, соответствуют ли даты начала и окончания мероприятия нашим требованиям  
    if (start_dt - now).days < 7 and end_dt > now:  
       # если даты соответствуют, добавляем строку в датафрейм  
         df = df.append(fill_df(event), ignore_index = True)  

Вы наверняка обратили внимание на то, что в коде цикла используется функция, которую мы пока не рассматривали, — fill_df(). Данная функция принимает словарь, описывающий мероприятие в формате, который предлагает API KudaGo, а возвращает словарь, который мы будем добавлять в новую строку датафрейма df. Рассмотрим структуру исходного и требуемого словаря на примере:
Исходные данные 
{'id': 180885,
'dates': [{'start': 1565539200, 'end': 1565544600}],
'title': 'концерт «Бах в мажоре» в Кафедральном соборе святых Петра и Павла', 
'description': 'Дуэт органа и саксофона исполнит музыку гения барокко.', 
'price': 'от 400 до 1800 руб.', 
'site_url': 'https://kudago.com/msk/event/kontsert-bah-v-mazhore/'}
Требуемый результат
{'id': 180885,
'title': 'концерт «Бах в мажоре» в Кафедральном соборе святых Петра и Павла',
'description': 'Дуэт органа и саксофона исполнит музыку гения барокко.',
'price': 'от 400 до 1800 руб.',
'start_date': '11-08-2019',
'end_date': '11-08-2019',
'url': 'https://kudago.com/msk/event/kontsert-bah-v-mazhore/'}

In [27]:
event_new={}
def clear_description(description):
    description = description.replace('<p>', '')
    description = description.replace('</p>', '')
    return description
def find_dates(event):
    startdate_dt= datetime.datetime.fromtimestamp(event[0]['start'])
    enddate_dt= datetime.datetime.fromtimestamp(event[0]['end'])
    startdate_txt = start_dt.strftime('%d-%m-%Y')
    enddate_txt = end_dt.strftime('%d-%m-%Y')
    return startdate_dt, enddate_dt, startdate_txt, enddate_txt
def fill_df(event):
    event_new['id'] = event['id']
    event_new['title'] = clear_description(event['title'])
    event_new['description'] = clear_description(event['description'])
    event_new['price'] = event['price']
    event_new['start_date'] = find_dates(event)[2]
    event_new['end_date'] = find_dates(event)[3]
    event_new['url'] = event['site_url']
    return event_new
df = pd.DataFrame(columns = 'id title description price start_date end_date url'.split())
for event in results:     
    # С помощью функции find_dates получаем данные о дате начала и окончания мероприятия в формате datetime  
    # т.к. даты в текстовом формате нам пока не нужны, оставляем на месте соответствующих переменных прочерки  
    startdate_dt,enddate_dt, startdate_txt, enddate_txt = find_dates(event)
    now = datetime.datetime.now()  
    # Проверяем, соответствуют ли даты начала и окончания мероприятия нашим требованиям  
    if (startdate_dt - now).days < 7 and enddate_dt > now:  
       # если даты соответствуют, добавляем строку в датафрейм  
         df = df.append(fill_df(event), ignore_index = True)  


KeyError: 0